In [55]:
import json
json_file = "/Users/admin/Working/nckh-chatbot-neo4j/crawl_data/cleaned_links4_haui.json"
with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Loop through each entry in the JSON file
web_test_set_urls = []
for entry in data:
    url = entry.get("url")
    web_test_set_urls.append(url)
    
from utils.load_html_test import load_document_web

docs = load_document_web(web_test_set_urls[30:70])
docs

[Document(metadata={'source': 'https://cpr.haui.edu.vn', 'title': 'TRUNG TÂM TRUYỀN THÔNG VÀ QUAN HỆ CÔNG CHÚNG - ĐẠI HỌC CÔNG NGHIỆP HÀ NỘI || HaUI', 'description': 'Trung tâm Truyền thông và Quan hệ công chúng là đơn vị trực thuộc Trường Đại học Công nghiệp Hà Nội, có chức năng tổ chức thực hiện các công việc: tổ chức sự kiện; quản lý thông tin và hoạt động truyền thông của Trường, xây dựng và quảng bá hình ảnh, thương hiệu của Trường Đại học Công nghiệp Hà Nội.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTRUNG TÂM TRUYỀN THÔNG VÀ QUAN HỆ CÔNG CHÚNG - ĐẠI HỌC CÔNG NGHIỆP HÀ NỘI || HaUI\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTrang HaUI\n\n\nPhòng\n\nPhòng Tổ chức hành chính\nPhòng Đào tạo\nPhòng Công tác sinh viên\nPhòng Khoa học công nghệ\nPhòng hợp tác đối ngoại\nPhòng Tài chính kế toán\nPhòng Quản trị\nPhòng Thanh tra Giáo dục\n\n\n\n\n\nKhoa\n\nKhoa Công nghệ Hoá\nKhoa Công nghệ may & TKTT\nKhoa Công nghệ thông tin\nKhoa Đ

In [56]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings.base import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from llm.get_llm import generate_test_model_function, get_embedding_function, get_model_function
generator_llm = LangchainLLMWrapper(get_model_function())
generator_embeddings = LangchainEmbeddingsWrapper(get_embedding_function())

from ragas.testset.persona import Persona

personas = [
    Persona(
        name="sinh viên tò mò",
        role_description="Một sinh viên Việt Nam đang tiến hành nghiên cứu về các nội dung liên quan đến hoạt động của Khoa công nghệ thông tin Trường Đại học Công nghiệp Hà Nội. Các nội dung liện quan tới các ngành, các hoạt động, các cuộc thi, các câu lạc bộ. Câu hỏi sẽ ở ngôn ngữ tiếng Việt",
    ),
    Persona(
        name="Giảng viên tận tâm",
        role_description="Một giảng viên tại Khoa Công nghệ Thông tin Trường Đại học Công nghiệp Hà Nội, luôn tìm kiếm cách nâng cao chất lượng giảng dạy và hỗ trợ sinh viên. Quan tâm đến các hoạt động học thuật, đổi mới phương pháp giảng dạy, và phát triển các chương trình hợp tác với doanh nghiệp. Câu hỏi sẽ ở ngôn ngữ tiếng Việt"
    ),
    Persona(
        name="Học sinh đang tìm hiểu",
        role_description="Một học sinh lớp 12 tại Việt Nam đang cân nhắc đăng ký vào Đại học Công nghiệp Hà Nội, đặc biệt quan tâm đến ngành Công nghệ Thông tin. Mong muốn tìm hiểu về chương trình đào tạo, cơ hội nghề nghiệp sau khi tốt nghiệp, và các tiêu chí xét tuyển của trường. Câu hỏi sẽ ở ngôn ngữ tiếng Việt"
    ),
    Persona(
        name="Phụ huynh lo lắng",
        role_description="Một phụ huynh có con đang chuẩn bị xét tuyển vào Đại học Công nghiệp Hà Nội. Quan tâm đến môi trường học tập, chất lượng đào tạo, cơ hội việc làm sau khi tốt nghiệp, cũng như chi phí học tập và hỗ trợ sinh viên tại trường. Câu hỏi sẽ ở ngôn ngữ tiếng Việt"
    ),


]

get llm 
get_embedding_function


In [57]:
from ragas.testset import TestsetGenerator
from ragas.cost import get_token_usage_for_openai

generator = TestsetGenerator(
    llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas
)

from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)
from ragas.testset.synthesizers.multi_hop.specific import MultiHopSpecificQuerySynthesizer

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
    (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
]

# for query, _ in distribution:
    # prompts = await query.adapt_prompts("spanish", llm=generator_llm)
    # query.set_prompts(**prompts)
    # query.save_prompts("/Users/admin/Working/thaibinh-chatbot/evaluate")

In [58]:
from ragas.cost import get_token_usage_for_openai

dataset = generator.generate_with_langchain_docs(
    docs,
    testset_size=80,
    query_distribution=distribution,
    
)

Applying SummaryExtractor:  60%|██████    | 32/53 [00:05<00:02,  8.59it/s]  Property 'summary' already exists in node '7cb323'. Skipping!
Property 'summary' already exists in node '72993e'. Skipping!
Applying SummaryExtractor:  72%|███████▏  | 38/53 [00:07<00:02,  5.20it/s]Property 'summary' already exists in node '13ac55'. Skipping!
Property 'summary' already exists in node '44052a'. Skipping!
Property 'summary' already exists in node '21e3b7'. Skipping!
Applying SummaryExtractor:  79%|███████▉  | 42/53 [00:07<00:01,  9.39it/s]Property 'summary' already exists in node 'b6c2d6'. Skipping!
Property 'summary' already exists in node 'b07614'. Skipping!
Applying SummaryExtractor:  98%|█████████▊| 52/53 [00:10<00:00,  2.79it/s]Property 'summary' already exists in node '5483fd'. Skipping!
Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/135 [00:00<?, ?it/s]Property 'summary_embedding' already exists in node '72993e'. Skipping!
Property 'summary_embedding' alre

In [59]:

eval_dataset = dataset.to_evaluation_dataset()
eval_dataset.to_csv("test_set_with_web_Haui_20-12_4.csv")